# Caso 3: Good Reads

## Primera Parte: Análisis Cuantitativo.

Primer examen preliminar de los datos.
* ¿En qué formato está el dataset?
* ¿Cómo podemos leerlo correctamente?
* ¿Qué campos hay en cada fichero del dataset?
* ¿Cuál es su significado?
* ¿Existen valores aparentemente incorrectos?

In [ ]:
!pip install -r ./../requirements.txt

In [ ]:
import geopandas as gpd
import folium
import os
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from caso03.descomprimir_dataset import unzip_dataset

## **Segunda Parte: Análisis Cualitativo**

* ¿Cuál fue el año en el que se publicaron más libros? Muéstralo en un gráfico
* ¿Y el autor más prolífico?
* ¿Cuántos libros suyos hay en el dataset?


In [ ]:
# Carga de datos
# Cargar dataset y descomprimir en /datos
unzip_dataset("./dataset/BookCrossing.zip","./data")

In [ ]:
df_books = pd.read_csv(os.path.abspath("./data/BookCrossing/BX-Books.csv"), delimiter=';', encoding='latin1', on_bad_lines='skip', low_memory=False)
df_books.drop(columns=['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], inplace=True)
df_books.head(5)

In [ ]:
df_ratings = pd.read_csv(os.path.abspath("./data/BookCrossing/BX-Book-Ratings.csv"), delimiter=';', na_values='NULL', encoding='latin1')
df_ratings.head(5)

In [ ]:
df_users = pd.read_csv(os.path.abspath("./data/BookCrossing/BX-Users.csv"), delimiter=';', na_values='NULL', encoding='latin1')
df_users.head(3)
# df_users.shape

### Cleaning dataset

In [ ]:
df_books['Year-Of-Publication'].unique()

In [ ]:
# Convert 'Year of Publication' to numeric, coercing errors to NaN
df_books['Year-Of-Publication'] = pd.to_numeric(df_books['Year-Of-Publication'], errors='coerce')
df_books_cleaned = df_books[(df_books['Year-Of-Publication'] >= 1900) & (df_books['Year-Of-Publication'] <= 2024)]
df_books_cleaned

In [ ]:
books = pd.read_csv(os.path.abspath("./data/BookCrossing/Books.csv"), delimiter=';', encoding='latin1', on_bad_lines='skip', low_memory=False)
books

In [ ]:
def clean_text(cell_value) -> str:
    text = str(cell_value)                      # Convert the input text to string
    text = text.strip()
    text = text.lower()                         # Convert to lowercase
    text = re.sub(r'\s+', ' ', text)            # replace repeated blanks with a single one
    text = re.sub(r' ', '-', text)              # replace blanks with '-'
    text = re.sub(r'[^a-zA-Z0-9\-]', '', text)  # Remove special characters 
    return text

In [ ]:
# Limpiamos textos
for column_name in ['Book-Title', 'Book-Author', 'Publisher']:
    df_books_cleaned[column_name] = df_books_cleaned[column_name].map(clean_text)
    
df_books_cleaned.head(3)

### **2.1 ¿Cuál fue el año en el que se publicaron más libros? Muéstralo en un gráfico ¿Y el autor más prolífico? ¿Cuántos libros suyos hay en el dataset?**


In [ ]:
# Agrupar por 'Year' y contar el número de publicaciones por cada año
yearly_publications = df_books_cleaned.groupby('Year-Of-Publication').size().reset_index(name='Count')
yearly_publications['Year-Of-Publication'] = yearly_publications['Year-Of-Publication'].astype(int)

# Encontrar el año con el mayor número de publicaciones
max_publication_year = yearly_publications.loc[yearly_publications['Count'].idxmax()]
print(max_publication_year)

In [ ]:
years = yearly_publications['Year-Of-Publication']
min_year = years.min()
max_year = years.max()

# Generate a range of years with a step of 5
years_5_step = range(min_year, max_year + 1, 5)

# Plot
plt.figure(figsize=(9, 7))
plt.barh(yearly_publications['Year-Of-Publication'], yearly_publications['Count'], color='skyblue')
plt.xlabel('Number of Publications')
plt.ylabel('Year')
plt.title('Number of Publications per Year')
plt.grid(axis='x', linestyle='--', alpha=0.7)

# Set y-ticks to show every 5 years
plt.yticks(years_5_step)

plt.show()

In [ ]:
# Agrupar por 'Author' y contar el número de publicaciones por cada autor
author_publications = df_books_cleaned.groupby('Book-Author').size().reset_index(name='Count')

# Encontrar el autor con el mayor número de publicaciones
most_prolific_author = author_publications.loc[author_publications['Count'].idxmax()]
print(most_prolific_author)

## **2.2 Analiza los orígenes geográficos y la edad de los reseñadores**

In [ ]:
# Extraer el país de la columna 'Location'
df_users['Country'] = df_users['Location'].apply(lambda x: x.split(',')[-1].strip() if pd.notnull(x) else x)

# Mostrar los primeros valores únicos de 'Country'
df_users['Country'].unique()

In [ ]:
# Estadísticas descriptivas de la edad
age_stats = df_users['Age'].describe()
age_stats

In [ ]:
df_users['Age'].unique()

In [ ]:
# Filtrar edades válidas
valid_ages = df_users[(df_users['Age'] > 0) & (df_users['Age'] <= 100)]

# Mostrar estadísticas de edades válidas
valid_age_stats = valid_ages['Age'].describe()
valid_age_stats

In [ ]:
valid_ages

In [ ]:
# Histograma de edades
plt.figure(figsize=(10, 6))
sns.histplot(data=valid_ages, x='Age', bins=30, kde=True)
plt.title('Distribución de Edades de los Reseñadores')
plt.xlabel('Edad')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# Aggregate the user counts by country
country_counts = df_users['Country'].value_counts().reset_index()
country_counts.columns = ['Country', 'User_Count']

# Step 2: Load a world map
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Convert country names to lowercase to match with the dataset
world['name'] = world['name'].str.lower()

# Step 3: Merge the user data with the world map
merged = world.merge(country_counts, how='left', left_on='name', right_on='Country')

In [ ]:
# Create the map
m = folium.Map(location=[20, 0], zoom_start=2)

# Add country data
folium.Choropleth(
    geo_data=merged,
    name='choropleth',
    data=merged,
    columns=['name', 'User_Count'],
    key_on='feature.properties.name',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='User Count'
).add_to(m)

# Add a layer control panel to the map
folium.LayerControl().add_to(m)

# Save map to an HTML file
m.save('user_distribution_map.html')
# Display the map
m